In [1]:

import requests
from bs4 import BeautifulSoup
# import streamlit as st
from transformers import pipeline
import re
import pymongo
from pymongo import MongoClient



# specify the URL of the news website to scrape
url = "https://www.theverge.com/"

# send a GET request to the website URL and store the response
response = requests.get(url)

# parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# find all the news article links on the website
article_links = soup.find_all("a", class_="group-hover:shadow-underline-franklin")

l = []
links = []

# loop through each article link and scrape the article text
for link in article_links:
    # extract the URL of the article
    article_url = link["href"]
    
    # send a GET request to the article URL and store the response
    article_response = requests.get(url + article_url)
    links.append(url + article_url)

#     # parse the HTML content of the article using BeautifulSoup
    article_soup = BeautifulSoup(article_response.content, "html.parser")
    
#     # find the main text content of the article and print it
    article_text = article_soup.find("div", class_="clearfix").get_text()
    l.append(article_text)

def get_first_thousand_words(text):
    words = re.findall(r'\b\w+\b|[^\w\s]', text)
    return ' '.join(words[:800])

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = []
for i in l:
    i = get_first_thousand_words(i)

    summaries.append(summarizer(i, max_length=300, min_length=100))

summariesText = [i[0]["summary_text"] for i in summaries]

In [40]:
links

['https://www.theverge.com//2023/3/24/23653377/ai-chatbots-comparison-bard-bing-chatgpt-gpt-4',
 'https://www.theverge.com//23630167/pacific-crest-trail-hiking-backpacking-gear-2600-miles',
 'https://www.theverge.com//2023/3/24/23654817/energy-efficiency-standards-doe-window-ac-air-purifier',
 'https://www.theverge.com//2023/3/24/23655071/ford-electric-truck-next-gen-project-t3-blueoval-city',
 'https://www.theverge.com//2023/3/24/23654710/ps5-xbox-console-resident-evil-jabra-elite-active-deal-sale']

In [2]:
summariesText

['The chatbots are out in force , but which is better and for what task ? We ’ve compared Google ’ s Bard, Microsoft’s Bing , and OpenAI ’s ChatGPT models with a range of questions. These questions span common requests from holiday tips to gaming advice to mortgage calculations. Bing is the winner here , but mainly because it picks one accurate hint ( Malenia is vulnerable to bleed damage ) and repeats it like Garth Marenghi doing a book reading. Bard is the only one to offer any help withMalenia ’S hellish Waterfowl Dance move ( although I don’t think it is the strongest strategy ) or advice for using a specific item.',
 'The Pacific Crest Trail spans around 2 , 650 miles through California, Oregon, and Washington. Hikers attempting to “ thru - hike ” a trail need to pack the gear necessary to stay alive. At roughly 11 pounds, my pack is way too heavy for the true ultralight purists. It omits some comfort items — a pillow, camp shoes, sleep clothes , and a midlayer / fleece — that mor

In [59]:
import sqlite3

conn.close()
conn = None


# Connect to the database
conn = sqlite3.connect('NewsSummarizer.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Delete all existing rows from the 'verge' table
sql_command = "DELETE FROM verge"
cursor.execute(sql_command)

# SQL command to insert sample data into the 'verge' table
sql_command = "INSERT INTO verge (summary, link) VALUES (?, ?)"

# Execute the SQL command for each row of sample data
for i in range(len(summariesText)):
    row = (str(summariesText[i])[2:-3], links[i],)
    cursor.execute(sql_command, row)

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


In [25]:
summariesText

[('The chatbots are out in force , but which is better and for what task ? We ’ve compared Google ’ s Bard, Microsoft’s Bing , and OpenAI ’s ChatGPT models with a range of questions. These questions span common requests from holiday tips to gaming advice to mortgage calculations. Bing is the winner here , but mainly because it picks one accurate hint ( Malenia is vulnerable to bleed damage ) and repeats it like Garth Marenghi doing a book reading. Bard is the only one to offer any help withMalenia ’S hellish Waterfowl Dance move ( although I don’t think it is the strongest strategy ) or advice for using a specific item.',),
 ('The Pacific Crest Trail spans around 2 , 650 miles through California, Oregon, and Washington. Hikers attempting to “ thru - hike ” a trail need to pack the gear necessary to stay alive. At roughly 11 pounds, my pack is way too heavy for the true ultralight purists. It omits some comfort items — a pillow, camp shoes, sleep clothes , and a midlayer / fleece — that

In [60]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('NewsSummarizer.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# SQL command to select table names
sql_command = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the SQL command
cursor.execute(sql_command)

# Fetch all the table names as a list of tuples
table_names = cursor.fetchall()

# Print the table names
print("Available tables:")
for table in table_names:
    print(table[0])

# Close the database connection
conn.close()


Available tables:
verge


In [80]:
# Connect to the database
conn = sqlite3.connect('NewsSummarizer.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Select all rows from the 'verge' table
sql_command = "SELECT * FROM verge"
cursor.execute(sql_command)

summaries_final = []
links_final = []
# Fetch all rows and print them
rows = cursor.fetchall()
for row in rows:
    summaries_final.append(row[0])
    links_final.append(row[1])    

# Close the database connection
conn.close()


In [81]:
summaries_final

['The chatbots are out in force , but which is better and for what task ? We ’ve compared Google ’ s Bard, Microsoft’s Bing , and OpenAI ’s ChatGPT models with a range of questions. These questions span common requests from holiday tips to gaming advice to mortgage calculations. Bing is the winner here , but mainly because it picks one accurate hint ( Malenia is vulnerable to bleed damage ) and repeats it like Garth Marenghi doing a book reading. Bard is the only one to offer any help withMalenia ’S hellish Waterfowl Dance move ( although I don’t think it is the strongest strategy ) or advice for using a specific item.',
 'The Pacific Crest Trail spans around 2 , 650 miles through California, Oregon, and Washington. Hikers attempting to “ thru - hike ” a trail need to pack the gear necessary to stay alive. At roughly 11 pounds, my pack is way too heavy for the true ultralight purists. It omits some comfort items — a pillow, camp shoes, sleep clothes , and a midlayer / fleece — that mor

In [67]:
summaries = []
links = []
for i in fromDB:
    summaries.append(i[0])
    links.append(i[1])

In [71]:
links

['https://www.theverge.com//2023/3/24/23653377/ai-chatbots-comparison-bard-bing-chatgpt-gpt-4',
 'https://www.theverge.com//23630167/pacific-crest-trail-hiking-backpacking-gear-2600-miles',
 'https://www.theverge.com//2023/3/24/23654817/energy-efficiency-standards-doe-window-ac-air-purifier',
 'https://www.theverge.com//2023/3/24/23655071/ford-electric-truck-next-gen-project-t3-blueoval-city',
 'https://www.theverge.com//2023/3/24/23654710/ps5-xbox-console-resident-evil-jabra-elite-active-deal-sale']